In [1]:
from PyDictionary import PyDictionary

dictionary=PyDictionary()

word = "death"


In [2]:
with open('saved_objects/top_words.txt') as tf:
    top_common_words = [line.strip() for line in tf.readlines()]
print(len(top_common_words))

200


In [3]:
print (dictionary.meaning(word))

{'Noun': ['the event of dying or departure from life', 'the permanent end of all life functions in an organism or part of an organism', 'the absence of life or state of being dead', 'the time when something ends', 'the time at which life ends; continuing until dead', 'the personification of death', 'a final state', 'the act of killing']}


/Users/edwinagnew/opt/anaconda3/envs/poetix/lib/python3.7/site-packages/PyDictionary/utils.py:5: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 5 of the file /Users/edwinagnew/opt/anaconda3/envs/poetix/lib/python3.7/site-packages/PyDictionary/utils.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  return BeautifulSoup(requests.get(url).text)


In [4]:
d = dictionary.meaning(word)
summary = word
for a in d['Noun'][:min(3,len(d['Noun']))]: #update for more pos's?
    words = a.split(" ")
    i = 0
    while words[i] in ["a", "the", word, "is"]: i+=1
    summary += " " + words[i]
    while words[-i] in ["a", "the", word, "is"]: i+=1
    summary += " " + words[-i]
print(summary)

death event life permanent organism absence dead


In [5]:
from transformers import BertTokenizer,BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.eval()


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
import torch

tokens = tokenizer.tokenize(summary)
print(tokens)
input_ids = torch.tensor(tokenizer.encode(tokens)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
predictions = outputs[0][0]
print(predictions.shape)
predictions

['death', 'event', 'life', 'permanent', 'organism', 'absence', 'dead']
torch.Size([9, 30522])


tensor([[ -8.0106,  -7.9767,  -7.8921,  ...,  -7.1867,  -7.5426,  -5.5443],
        [-12.4375, -12.6186, -12.5716,  ..., -10.1929, -10.9603, -11.3200],
        [ -8.0563,  -8.2167,  -8.1880,  ...,  -6.4818,  -9.0173,  -6.9412],
        ...,
        [ -6.7635,  -6.8820,  -6.7377,  ...,  -5.9392,  -7.0242,  -6.9825],
        [ -8.4582,  -8.8511,  -8.9916,  ...,  -7.6178, -10.0740,  -3.9625],
        [-13.8706, -13.8998, -13.7247,  ..., -11.3769, -12.4060,  -8.9048]],
       grad_fn=<SelectBackward>)

In [7]:
average = torch.mean(predictions[1:-1,1:-1], axis = 0) #####IMPROVE THIS
from heapq import nlargest
indexes = torch.arange(0,len(average))
n = nlargest(10, indexes, key=lambda i: average[i])
print(n)
tokenizer.convert_ids_to_tokens(n)

[tensor(1011), tensor(1996), tensor(1009), tensor(2202), tensor(2330), tensor(1010), tensor(1998), tensor(2165), tensor(2723), tensor(1036)]


['-', 'the', '+', 'take', 'open', ',', 'and', 'took', 'floor', '`']

In [8]:
import nltk
def is_good_word(str):
    punctuation = ['-', ':', '+', ',', '～', '`', '!', '.', ',', '?', '*']
    if str in punctuation or '#' in str:
        return False
    if str in top_common_words:
        return False
    pos = nltk.pos_tag([str])[0][1]
    if 'NN' in pos: #or 'JJ' in pos or 'RB' in pos or 'VB' in pos: #if its a noun, adjective(? or adverb we good just noun for now
        return True
    return False

In [9]:
#with torch.no_grad():
average = torch.mean(predictions[1:-1,1:-1], axis = 0) ##improve this!!!
x = sorted(average)
k = 1
best = torch.argmax(average)
while not is_good_word(tokenizer.convert_ids_to_tokens(best.item())):
    k+=1
    y = x[-k]
    best = (average == y).nonzero().flatten()
print(best)
print("The metaphor is", tokenizer.convert_ids_to_tokens(best.item()))

tensor([2723])
The metaphor is floor


In [101]:
#tokenizer.convert_ids_to_tokens(nltk.pos_tag([1024])[0][1]) == ':'
#nltk.pos_tag([tokenizer.convert_ids_to_tokens(1024)])[0][1] == ':'

True

In [11]:
tokenizer.convert_ids_to_tokens(2050)

'##a'

In [10]:
'#' in '##a'
nltk.pos_tag(["open"])

[('open', 'JJ')]